In [ ]:
# Install Gradio and required LangChain packages
# !pip install -U PyPDF2 gradio langchain langchain-community langchain-core langchain-huggingface

**Imports and Setup**

In [1]:
import gradio as gr
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.runnables import Runnable
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_ollama.llms import OllamaLLM
from langchain.text_splitter import RecursiveCharacterTextSplitter

from PyPDF2 import PdfReader
from PIL import Image
import pytesseract
import cv2
import tempfile
from pathlib import Path
import os

**Load Embeddings and VectorStores**

In [2]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

faiss_docs = FAISS.load_local(
    folder_path="../vectorstores/faiss_index_multimodal/faiss_docs",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

faiss_images = FAISS.load_local(
    folder_path="../vectorstores/faiss_index_multimodal/faiss_images",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

**Define the RAG Chain**

In [12]:
llm = OllamaLLM(model="deepseek-r1:7b")  # Can switch to another Ollama-compatible LLM

prompt_template = PromptTemplate.from_template(
    """
        You are an assistant for municipal issue queries in Singapore.

        Answer the question below using only the provided context.
        If possible, in your reply, provide the user as much details as possible.
        If unsure, say "I don't know".

        Context:
        {context}

        Question:
        {question}

        Format your reply starting with "**Answer:**" followed by your clear response.
    """
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain: Runnable = (
    {
        "context": lambda q: format_docs(faiss_docs.similarity_search(q, k=5) + faiss_images.similarity_search(q, k=3)),
        "question": lambda q: q
    }
    | prompt_template
    | llm
)


**File Upload Processing (PDF/Image)**

In [18]:
def extract_text_from_pdf(path):
    try:
        reader = PdfReader(path)
        return "\n".join(page.extract_text() or "" for page in reader.pages)
    except:
        return ""

def extract_text_from_image(path):
    try:
        image = cv2.imread(path)
        if image is None:
            print(f"[WARN] Failed to load image from {path}")
            return ""
        text = pytesseract.image_to_string(image)
        print(f"[DEBUG] OCR result:\n{text[:300]}")
        return text
    except Exception as e:
        print(f"[ERROR] OCR failed for {path}: {e}")
        return ""

def process_upload(file_path):
    ext = Path(file_path).suffix.lower()
    if ext == ".pdf":
        content = extract_text_from_pdf(file_path)
    elif ext in [".png", ".jpg", ".jpeg"]:
        content = extract_text_from_image(file_path)
    else:
        return False

    if content:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50,
            separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
        )
        chunks = text_splitter.split_text(content)
        faiss_images.add_texts(chunks)
        return True
    return False

**Multimodal RAG Logic**

In [16]:
def extract_final_answer(result: str) -> str:
    if "**Answer:**" in result:
        return result.split("**Answer:**")[-1].strip()
    return result.strip()

In [19]:
import gradio as gr
from pathlib import Path

def run_multimodal_rag(message=None, file=None, chat_history=None):
    query = message.strip() if message else ""
    file_text = ""

    if file and isinstance(file, str) and Path(file).exists():
        ext = Path(file).suffix.lower()
        try:
            if ext == ".pdf":
                file_text = extract_text_from_pdf(file)
            elif ext in [".png", ".jpg", ".jpeg"]:
                file_text = extract_text_from_image(file)
        except Exception as e:
            file_text = f"[ERROR] Cannot extract text: {e}"

        if file_text.strip():
            query += f"\n\n[Context from uploaded file]:\n{file_text.strip()}"

    if not query.strip():
        return [chat_history, "", chat_history]

    try:
        # Assume `rag_chain.invoke(query)` gives us the assistant's reply
        result = rag_chain.invoke(query)
        reply = extract_final_answer(result)
        chat_history = chat_history or []
        chat_history.append({"role": "user", "content": message})
        chat_history.append({"role": "assistant", "content": reply})
        return [chat_history, "", chat_history]
    except Exception as e:
        chat_history.append({"role": "user", "content": message})
        chat_history.append({"role": "assistant", "content": "Error: " + str(e)})
        return [chat_history, "", chat_history]


**Gradio UI**

In [6]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\Fleming Siow\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"

In [21]:
import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=400, show_copy_button=True, type="messages")
    state = gr.State([])  # Stores chat history

    with gr.Row():
        with gr.Column(scale=4):
            txt = gr.Textbox(
                placeholder="Ask something like: 'What are the rules for displaying banners in Clementi?'",
                show_label=False
            )
        with gr.Column(scale=1):
            upload = gr.File(
                file_types=[".pdf", ".png", ".jpg", ".jpeg"],
                file_count="single",
                type="filepath",
                label="Upload a document or image"
            )

    btn = gr.Button("Send")

    btn.click(
        run_multimodal_rag,
        inputs=[txt, upload, state],
        outputs=[chatbot, txt, state],
        api_name="chat"
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


[DEBUG] OCR result:
HOUSING &
DEVELOPMENT
BOARD

INDEMNITY FORM
(RENOVATION)

HDB Branch

RENOVATIONS TO BE CARRIED OUT AT APT BLK

In consideration of your agreeing at my/our request fo grant a renovation permitto carry out renovations.
specified in the renovation permit at Apt Bik .
vwe NAIC and.

NAIC the proposed p
